# Problem Set 4



Team: 
    Lea Schmierer 3546563;
    Angelina Basova 3704658;
    Daniel Knorr 3727033

## Exercise 1


## Exercise 2

#### a) Explain briefly why all nodes in the clique have the same PageRank value.

All nodes in the clique have the same value, because they all have the same number of in-links and out-links, therefore none of these nodes are more important than the others. Every note is weighted the same.



#### b) 
The Google Matrix A: A= beta*M+(1-beta)[1/N] with beta = probability to follow a link at random

Now: beta= 1 ??????? -> ODER???? oder ist 1-beta= 1????????



#### c)


#### d) Explain briefly why random teleports not only solve the spider trap problem but also the dead ends problem. 

## Exercise 3

## Exercise 4

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, IntegerType


######################
# 4a)
print("Exercise 4a)")
######################

sparkSession = SparkSession.builder.appName('A4E4').getOrCreate()

print("# --------------------------------")
print("# Create DataFrame DF1 webStanfordDF")
print("# --------------------------------")

rdd_datafile = sparkSession.sparkContext.textFile('./web-Stanford_small.txt')

rdd_web_standford = rdd_datafile.map(lambda x: x.split("\t"))

webStanfordRDD = rdd_web_standford.map(lambda p: (
    int(p[0]), int(p[1].strip())))

schema = StructType([
    StructField("FromNodeId", IntegerType(), True),
    StructField("ToNodeId", IntegerType(), True)
])

webStanfordDF = sparkSession.createDataFrame(webStanfordRDD, schema)

# Get all Nodes as a Set
listFromNodeId = webStanfordDF.select(webStanfordDF["FromNodeId"]).toPandas()['FromNodeId']
listeToNodeId = webStanfordDF.select(webStanfordDF["ToNodeId"]).toPandas()['ToNodeId']

listFromNodeId = list(listFromNodeId)
listeToNodeId = list(listeToNodeId)

allNodesList = listFromNodeId + listeToNodeId
allNodesSet = set(allNodesList)

# Define dictionary
dic = {}

def generateDictionary(allNodesSet, webStanfordDF, dic):
    for node in allNodesSet:
        liste = []
        filteredByFromNodeId = webStanfordDF.filter(webStanfordDF["FromNodeId"] == node)
        listOutgoingLinksTo = filteredByFromNodeId.select(filteredByFromNodeId["ToNodeId"]).toPandas()['ToNodeId']
        filteredByToNodeId = webStanfordDF.filter(webStanfordDF["ToNodeId"] == node)
        listIngoingLinksFrom = filteredByToNodeId.select(filteredByToNodeId["FromNodeId"]).toPandas()[
            'FromNodeId']
        listOutgoingLinksTo = list(listOutgoingLinksTo)
        listIngoingLinksFrom = list(listIngoingLinksFrom)
        liste.append(listOutgoingLinksTo)
        liste.append(listIngoingLinksFrom)
        dic[node] = liste


generateDictionary(allNodesSet, webStanfordDF, dic)

print("Dictionary Composition: Node i: [<Out-neighbors of i>],<[In-neighbors of i]>")
print(dic)